In [97]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [98]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py"
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py"
eval_name = "eval_res_reeval.json"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, eval_name)):
        with open(os.path.join(sub_folder_path, eval_name)) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        trials.append((params, eval_res, sub_folder_path))

In [99]:
# add 0 iterations if not present
for i, (params, eval_res, sub_folder_path) in enumerate(trials):
    if "iterations" not in eval_res["cummulative_stats"]:
        cum_stats_dict = eval_res["cummulative_stats"]
        cum_stats_dict["iterations"] = {}
        cum_stats_dict["iterations"]["mean"] = 0
        cum_stats_dict["iterations"]["std"] = 0
        eval_res["cummulative_stats"] = cum_stats_dict
        trials[i] = (params, eval_res, sub_folder_path)



In [100]:
trials[0][0].keys()

dict_keys(['batch_size', 'data_aug_factor', 'edge_threshold_val', 'grace_epochs', 'hidden_dim', 'loss_on_full_matrix', 'lr', 'lr_cooldown', 'lr_factor', 'lr_min', 'lr_patience', 'lr_threshold', 'message_net_dropout', 'message_net_layers', 'message_passing_steps', 'num_epochs', 'weight_decay'])

In [101]:
# trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
# trials.sort(key=lambda x:x[0]["edge_threshold_val"])#["iterations"]["mean"])
for trial in trials: 
    print(trial[0])
    print(trial[1]["cummulative_stats"])
    print("---")

{'batch_size': 32, 'data_aug_factor': 1.9081436498666626, 'edge_threshold_val': 2.111290729660567, 'grace_epochs': 5, 'hidden_dim': 256, 'loss_on_full_matrix': True, 'lr': 0.00026128510321670335, 'lr_cooldown': 2, 'lr_factor': 0.5, 'lr_min': 1e-06, 'lr_patience': 3, 'lr_threshold': 0.001, 'message_net_dropout': 0.2191115137292724, 'message_net_layers': 5, 'message_passing_steps': 3, 'num_epochs': 30, 'weight_decay': 0.0002974629996807899}
{'energy_abs': {'mean': 1.4001809385812476, 'std': 2.257128929105722}, 'iterations': {'mean': nan, 'std': nan}, 'energy_rel': {'mean': 0.0007876300764178357, 'std': 0.0012884871990914333}, 'diis': {'mean': 0.04228943417783732, 'std': 0.005634772089230802}, 'rmse': {'mean': 0.008967282765809922, 'std': 0.0005198778065816581}}
---
{'batch_size': 32, 'data_aug_factor': 2.32843109208835, 'edge_threshold_val': 2.046848789531119, 'grace_epochs': 5, 'hidden_dim': 256, 'loss_on_full_matrix': True, 'lr': 0.0001160861493167114, 'lr_cooldown': 2, 'lr_factor': 0.

In [102]:
table_data = []
Hartree_conv = 1000 # mH
for i, (params, metrics, path) in enumerate(trials):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.3f}({float(metrics['energy_abs']['std']):.3f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.4f}({float(metrics['energy_rel']['std']):.4f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.4f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)
    print(path)
import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00079_79_batch_size=32,data_aug_factor=1.9081,edge_threshold_val=2.1113,hidden_dim=256,lr=0.0003,message_net__2025-07-11_22-04-17
/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00070_70_batch_size=32,data_aug_factor=2.3284,edge_threshold_val=2.0468,hidden_dim=256,lr=0.0001,message_net__2025-07-11_20-30-26
/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00082_82_batch_size=32,data_aug_factor=1.6722,edge_threshold_val=2.1384,hidden_dim=256,lr=0.0004,message_net__2025-07-11_22-25-17
/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00009_9_batch_size=32,data_aug_factor=2.4478,edge_threshold_val=2.

In [103]:
table_data = []
for i, (params, metrics, path) in enumerate(trials):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 \\
Hidden Dimension & 256 & 256 & 256 & 128 & 128 & 256 \\
Batch Size & 32 & 32 & 32 & 32 & 32 & 16 \\
Data aug. & 1.91 & 2.33 & 1.67 & 2.45 & 1.50 & 2.08 \\
Edge thres. & 2.11 & 2.05 & 2.14 & 2.12 & 2.11 & 3.73 \\
Message p. steps & 3 & 3 & 3 & 2 & 4 & 2 \\
Message Net Dropout & 0.22 & 0.11 & 0.18 & 0.23 & 0.29 & 0.26 \\
Message Net Layers & 5 & 3 & 3 & 3 & 5 & 4 \\
Learn Rate & 2.61e-04 & 1.16e-04 & 3.54e-04 & 4.36e-03 & 3.97e-03 & 3.06e-04 \\
Weight Decay & 2.97e-04 & 2.35e-05 & 5.21e-05 & 4.43e-05 & 9.97e-04 & 2.35e-05 \\
\bottomrule
\end{tabular}

